In [1]:
"""
This is an example of how to use the linear learner algorithm (binary
classification).  It no longer works, as it is missing some things, such as
some AWS resources and input files, but it is being kept here for future
reference.
"""

# Install dependencies

%pip install boto3
%pip install sagemaker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [28]:
# Set up AWS session and other things

import os
import boto3
import re
import sagemaker

# Default session for SSO authentication
aws_profile = os.getenv(key="AWS_PROFILE")
if (aws_profile is not None):
    boto3.setup_default_session(profile_name=aws_profile)

# TODO: Create reusable sagemaker role in terraform
#role = sagemaker.get_execution_role()
role = "arn:aws:iam::876137462851:role/service-role/AmazonSageMaker-ExecutionRole-20230222T122399"

region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
bucket = sagemaker.Session().default_bucket()

prefix = (
    "sagemaker/stocks-test"  # place to upload training files within the bucket
)

In [29]:
# TODO: move imports to the cells where they're used maybe

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac

In [30]:
# Download, split and save temporary data files for training

s3 = boto3.client("s3")

filename = "orig_data.csv"
s3.download_file("sagemaker-eu-west-1-876137462851", "sagemaker/data.csv", filename)
data = pd.read_csv(
    filename,
    #header=None
)

# specify columns extracted from wbdc.names
data.columns = [
    "Action",
    "SMA50",
    "Close"
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

# print the shape of the data file
print(data.shape)

# show the top few rows
display(data.head())

# describe the data object
display(data.describe())

# we will also summarize the categorical field diganosis
display(data.Action.value_counts())

rand_split = np.random.rand(len(data))
train_list = rand_split < 0.6
val_list = (rand_split >= 0.6) & (rand_split < 0.8)
test_list = rand_split >= 0.8

data_train = data[train_list]
data_val = data[val_list]
data_test = data[test_list]

train_y = ((data_train.iloc[:, 0] == "buy") + 0).to_numpy()
train_X = data_train.iloc[:, 1:].to_numpy()

val_y = ((data_val.iloc[:, 0] == "buy") + 0).to_numpy()
val_X = data_val.iloc[:, 1:].to_numpy()

test_y = ((data_test.iloc[:, 0] == "buy") + 0).to_numpy()
test_X = data_test.iloc[:, 1:].to_numpy();

print(train_y)
print(train_X)
print(val_y)
print(val_X)
print(test_y)
print(test_X)

# Save temporary training files in bucket for training job

train_file = "linear_train.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, train_X.astype("float32"), train_y.astype("float32"))
f.seek(0)

s3.upload_fileobj(f, bucket, os.path.join(prefix, "train", train_file))

validation_file = "linear_validation.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, val_X.astype("float32"), val_y.astype("float32"))
f.seek(0)

s3.upload_fileobj(f, bucket, os.path.join(prefix, "validation", train_file))

(999, 3)


Action   SMA50   Close
0   sell  226.55  171.58
1   sell  105.76   30.57
2    buy  217.03  256.58
3    buy   90.31  144.21
4    buy   58.45  259.53

SMA50       Close
count  999.000000  999.000000
mean   149.429399  147.699540
std     87.641989   89.052482
min      0.390000    0.220000
25%     74.425000   68.490000
50%    148.590000  148.170000
75%    223.885000  225.415000
max    299.830000  299.510000

sell    504
buy     495
Name: Action, dtype: int64

In [60]:
# Create and run training job
 
from sagemaker import image_uris

container = image_uris.retrieve(region=boto3.Session().region_name, framework="linear-learner")

linear_job = "stocks-test2-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

print("Job name is:", linear_job)

linear_training_params = {
    "RoleArn": role,
    "TrainingJobName": linear_job,
    "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.c4.2xlarge", "VolumeSizeInGB": 10},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train/".format(bucket, prefix),
                    "S3DataDistributionType": "ShardedByS3Key",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
    ],
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/".format(bucket, prefix)},
    "HyperParameters": {
        "feature_dim": "2",
        "mini_batch_size": "10",
        "predictor_type": "regressor",
        "epochs": "10",
        "num_models": "32",
        "loss": "absolute_loss",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 60 * 60},
}

%%time

region = boto3.Session().region_name
sm = boto3.client("sagemaker")

sm.create_training_job(**linear_training_params)

status = sm.describe_training_job(TrainingJobName=linear_job)["TrainingJobStatus"]
print(status)
sm.get_waiter("training_job_completed_or_stopped").wait(TrainingJobName=linear_job)
if status == "Failed":
    message = sm.describe_training_job(TrainingJobName=linear_job)["FailureReason"]
    print("Training failed with the following error: {}".format(message))
    raise Exception("Training job failed")

Job name is: stocks-test2-2023-03-17-15-31-19


In [63]:
# Create model, endpoint config, and endpoint

linear_hosting_container = {
    "Image": container,
    "ModelDataUrl": sm.describe_training_job(TrainingJobName=linear_job)["ModelArtifacts"][
        "S3ModelArtifacts"
    ],
}

create_model_response = sm.create_model(
    ModelName=linear_job, ExecutionRoleArn=role, PrimaryContainer=linear_hosting_container
)

print(create_model_response["ModelArn"])

linear_endpoint_config = "stocks-test-endpoint-config-" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)
print(linear_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=linear_endpoint_config,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "ModelName": linear_job,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

%%time

linear_endpoint = "stocks-test-endpoint-" + time.strftime("%Y%m%d%H%M", time.gmtime())
print(linear_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=linear_endpoint, EndpointConfigName=linear_endpoint_config
)
print(create_endpoint_response["EndpointArn"])

resp = sm.describe_endpoint(EndpointName=linear_endpoint)
status = resp["EndpointStatus"]
print("Status: " + status)

sm.get_waiter("endpoint_in_service").wait(EndpointName=linear_endpoint)

resp = sm.describe_endpoint(EndpointName=linear_endpoint)
status = resp["EndpointStatus"]
print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

if status != "InService":
    raise Exception("Endpoint creation did not succeed")

stocks-test-endpoint-config-2023-03-17-15-37-00
Endpoint Config Arn: arn:aws:sagemaker:eu-central-1:876137462851:endpoint-config/stocks-test-endpoint-config-2023-03-17-15-37-00


In [65]:
# Run tests on real-time endpoint

def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=",", fmt="%g")
    return csv.getvalue().decode().rstrip()

runtime = boto3.client("runtime.sagemaker")

payload = np2csv(test_X)
response = runtime.invoke_endpoint(
    EndpointName=linear_endpoint, ContentType="text/csv", Body=payload
)
result = json.loads(response["Body"].read().decode())
print(result)
test_pred = np.array([r["score"] for r in result["predictions"]])
print(np.c_[np.c_[test_X, test_y], test_pred])

test_mae_linear = np.mean(np.abs(test_y - test_pred))
test_mae_baseline = np.mean(
    np.abs(test_y - np.median(train_y))
)  ## training median as baseline predictor

print("Test MAE Baseline :", round(test_mae_baseline, 3))
print("Test MAE Linear:", round(test_mae_linear, 3))

test_pred_class = (test_pred > 0.5) + 0
test_pred_baseline = np.repeat(np.median(train_y), len(test_y))

prediction_accuracy = np.mean((test_y == test_pred_class)) * 100
baseline_accuracy = np.mean((test_y == test_pred_baseline)) * 100

print("Prediction Accuracy:", round(prediction_accuracy, 1), "%")
print("Baseline Accuracy:", round(baseline_accuracy, 1), "%")

{'predictions': [{'score': 0.28333568572998047}, {'score': 0.7672820091247559}, {'score': 0.5573155879974365}, {'score': 0.1974848508834839}, {'score': 0.7810168862342834}, {'score': 0.621572732925415}, {'score': 0.9613379240036011}, {'score': 1.070367693901062}, {'score': 0.8595460057258606}, {'score': 1.4184136390686035}, {'score': 0.6494091153144836}, {'score': 1.1906051635742188}, {'score': 0.6733908653259277}, {'score': 0.0919843316078186}, {'score': 0.8074506521224976}, {'score': 0.7223159670829773}, {'score': 0.8166096210479736}, {'score': -0.12412238121032715}, {'score': 0.9714826345443726}, {'score': 1.2089651823043823}, {'score': 0.1377752423286438}, {'score': 0.5983049869537354}, {'score': 0.7687161564826965}, {'score': -0.14216995239257812}, {'score': 0.626885712146759}, {'score': 0.6257880926132202}, {'score': 1.0072132349014282}, {'score': 0.8501412868499756}, {'score': 0.6240132451057434}, {'score': 0.6169694662094116}, {'score': 0.8042582869529724}, {'score': 0.91685080

In [73]:
# Delete the model, endpoint and endpoint config resources that were created above

if any(e["EndpointName"] == linear_endpoint for e in sm.list_endpoints()["Endpoints"]):
    sm.delete_endpoint(EndpointName=linear_endpoint)
if any(ec["EndpointConfigName"] == linear_endpoint_config for ec in sm.list_endpoint_configs()["EndpointConfigs"]):
    sm.delete_endpoint_config(EndpointConfigName=linear_endpoint_config)
if any(m["ModelName"] == linear_job for m in sm.list_models()["Models"]):
    sm.delete_model(ModelName=linear_job)


{'Endpoints': [{'EndpointName': 'stocks-test-endpoint-202303171537', 'EndpointArn': 'arn:aws:sagemaker:eu-central-1:876137462851:endpoint/stocks-test-endpoint-202303171537', 'CreationTime': datetime.datetime(2023, 3, 17, 15, 37, 10, 627000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2023, 3, 17, 15, 40, 48, 646000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}], 'ResponseMetadata': {'RequestId': '25498e6c-a717-4a45-ac45-b387714d1629', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '25498e6c-a717-4a45-ac45-b387714d1629', 'content-type': 'application/x-amz-json-1.1', 'content-length': '268', 'date': 'Fri, 17 Mar 2023 15:58:56 GMT'}, 'RetryAttempts': 0}}
